In [ ]:
import execo
import execo_g5k
import execo_engine
import time
import os

# this will be /home/lmascare
HOME_DIR = os.path.expanduser("~")

PATH_TO_SIF_FILE = HOME_DIR + "/bench/docker/images/bench.sif"

SIMULATION_INI = HOME_DIR + "/bench/simulation/setup.ini"
PDI_DEISA_YML = HOME_DIR + "/bench/simulation/io_deisa.yml"
SIM_EXECUTABLE = HOME_DIR + "/bench/simulation/build/main"
DEISA_PATH = HOME_DIR + "/bench/deisa/"

ANALYTICS_PY_FILE = HOME_DIR + "/bench/in-situ/bench_deisa.py"

SCHEDULER_FILE = HOME_DIR + "/bench/experiment/scheduler.json"

OUTPUT_DIR = HOME_DIR + "/bench/experiment/"

NDASKWORKERS = 2
DASK_WORKERS_PER_NODE = 1


2025-05-20 13:46:13,665 WARNING: terminated: <SshProcess('singularity exec /home/lmascare/bench/docker/images/bench.sif bash -c "dask scheduler --host dahu-9.grenoble.grid5000.fr --scheduler-file /home/lmascare/bench/experiment/scheduler.json > /home/lmascare/bench/experiment/scheduler.e 2>&1"', Host('dahu-9.grenoble.grid5000.fr'), name=singularity exec /home/lmascare/benc..., started=True, start_date=2025-05-20 13:36:35+02:00, ended=True, end_date=2025-05-20 13:46:13+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=65280, ok=False, pid=3486719, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes', '-o', 'PasswordAuthentication=no', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'ConnectTimeout=20', 'dahu-9.grenoble.grid5000.fr', 'singularity exec /home/lmascare/bench/docker/images/bench.sif bash -c "dask scheduler --host dahu-9.grenoble.grid5000.fr --scheduler-file /home/lmascare/bench/experim

In [2]:
""" 
Get configs from a .ini file.

Arguments:
    config_file (str): Path to the .ini configuration file.
"""
def get_configs(config_file):
    import configparser

    config = configparser.ConfigParser()
    config.read(config_file)

    # Assuming the .ini file has sections and keys
    # Example: [section_name] key_name = value
    configs = {}
    for section in config.sections():
        for key, value in config.items(section):
            configs[key] = value

    return configs

In [3]:
configs = get_configs(SIMULATION_INI)

nb_reserved_nodes = 1

jobs = execo_g5k.oarsub(
    [
        (
            execo_g5k.OarSubmission(f"nodes={nb_reserved_nodes}", walltime=10*60),
            "grenoble",
        )
    ]
)

job_id, site = jobs[0]
print(f"Job {job_id} reserved on site {site}")

nodes = execo_g5k.oar.get_oar_job_nodes(job_id, site)
head_node, nodes = nodes[0], nodes[1:]

print(f"Head node: {head_node}")
print(f"Other nodes: {nodes}")


cores_per_node = execo_g5k.get_host_attributes(head_node)["architecture"]["nb_cores"]
total_simulation_cores = cores_per_node * len(nodes)

print(f"Total simulation cores: {total_simulation_cores}")

Job 2476681 reserved on site grenoble
Head node: Host('dahu-9.grenoble.grid5000.fr')
Other nodes: []
Total simulation cores: 0


In [4]:
##########################################################
#                   RUNNING SCHEDULER
##########################################################
# Sending the commands for head node

if os.path.exists(SCHEDULER_FILE):
    os.remove(SCHEDULER_FILE)

#redirecting the output to a file
scheduler_cmd = (
    f"dask scheduler "
    f"--host {head_node.address} "
    f"--scheduler-file {SCHEDULER_FILE} "
    # "--port 8786 "
    f"> {OUTPUT_DIR}scheduler.e 2>&1"
)
print(scheduler_cmd)
scheduler_process = execo.SshProcess(
    f'singularity exec {PATH_TO_SIF_FILE} bash -c "{scheduler_cmd}"',
    head_node,
)
scheduler_process.start()

# Wait for the scheduler to start by checking the scheduler file
while not os.path.exists(SCHEDULER_FILE):
    time.sleep(1)

dask scheduler --host dahu-9.grenoble.grid5000.fr --scheduler-file /home/lmascare/bench/experiment/scheduler.json > /home/lmascare/bench/experiment/scheduler.e 2>&1


In [5]:
##########################################################
#                   RUNNING WORKERS
##########################################################

# worker_cmd = (
#     f"dask worker tcp://{head_node.address}:8786 "
#     "--nworkers {DASK_WORKERS_PER_NODE} "
#     "--nthreads 1 "
#     "--local-directory /tmp "
#     "--scheduler-file={SCHEDULER_FILE} "
#     "> {OUTPUT_DIR}worker.e 2>&1"
# )

# # for node in nodes:
#     # redirecting the output to a file
# worker_process = execo.SshProcess(
#     f'singularity exec {PATH_TO_SIF_FILE} bash -c "{worker_cmd}"',
#     nodes[0],
# )
# worker_process.start()

worker_cmd = (
    f"dask worker "
    # "tcp://{head_node.address}:8786 "
    f"--nworkers {DASK_WORKERS_PER_NODE} "
    "--nthreads 1 "
    "--local-directory /tmp "
    f"--scheduler-file {SCHEDULER_FILE} "
    f"> {OUTPUT_DIR}worker.e 2>&1"
)

# for node in nodes:
    # redirecting the output to a file
worker_process = execo.SshProcess(
    f'singularity exec {PATH_TO_SIF_FILE} bash -c "{worker_cmd}"',
    head_node,
)
worker_process.start()

SshProcess('singularity exec /home/lmascare/bench/docker/images/bench.sif bash -c "dask worker --nworkers 1 --nthreads 1 --local-directory /tmp --scheduler-file /home/lmascare/bench/experiment/scheduler.json > /home/lmascare/bench/experiment/worker.e 2>&1"', Host('dahu-9.grenoble.grid5000.fr'))

In [6]:
##########################################################
#                   RUNNING ANALYTICS
##########################################################

# python3 $OPWD/in-situ/bench_deisa.py $NDASKWORKERS $SCHEFILE 2>analytics.e&

py_cmd = (
    'export PYTHONPATH=/home/lmascare/bench/deisa/:$PYTHONPATH; '
    f'python3 {ANALYTICS_PY_FILE} {NDASKWORKERS} {SCHEDULER_FILE} > {OUTPUT_DIR}analytics.e 2>&1'
)
analytics_cmd = (
    # 'hostname'
    f'singularity exec {PATH_TO_SIF_FILE} bash -c "{py_cmd}"'

)
print("Analytics command:")
print(analytics_cmd)

analytics_process = execo.SshProcess(
    analytics_cmd,
    head_node,
)
analytics_process.start()
# analytics_process.wait()

print()
print("#"*20)
print("Output:")
print(analytics_process.stdout)
print("-"*20)
print("Error:")
print(analytics_process.stderr)
print()
print("#"*20)


Analytics command:
singularity exec /home/lmascare/bench/docker/images/bench.sif bash -c "export PYTHONPATH=/home/lmascare/bench/deisa/:$PYTHONPATH; python3 /home/lmascare/bench/in-situ/bench_deisa.py 2 /home/lmascare/bench/experiment/scheduler.json > /home/lmascare/bench/experiment/analytics.e 2>&1"

####################
Output:

--------------------
Error:


####################


In [7]:
##########################################################
#                   RUNNING SIMULATION
##########################################################

# Starting the simulation 32 - 30 for sim and 2 for dask 

mx = int(configs["mx"])
my = int(configs["my"])
mz = int(configs["mz"])

MPI_NP = mx * my * mz

print(f"Running simulation with {MPI_NP} MPI processes")

host_list = ",".join([f"{node.address}:{2}" for node in nodes])
simulation_cmd = (
    # 'hostname'
    f'pdirun {SIM_EXECUTABLE} {SIMULATION_INI} {PDI_DEISA_YML} --kokkos-map-device-id-by=mpi_rank > {OUTPUT_DIR}simulation.e 2>&1'
)

# Build the command with two singularity exec calls:
mpi_cmd = (
    'export OMP_NUM_THREADS=2; '
    'export OMP_PROC_BIND=spread; '
    'export OMP_PLACES=threads; '
    f"mpirun -np {MPI_NP} "
    # f"--host {host_list} "
    f"singularity exec {PATH_TO_SIF_FILE} {simulation_cmd}"
)

print("Simulation command:")
print(mpi_cmd)
mpi_process = execo.SshProcess(
    mpi_cmd,
    head_node,
)
mpi_process.start()

mpi_process.wait()

print("Output:")
print(mpi_process.stdout)
print("-"*20)
print("Error:")
print(mpi_process.stderr)
print()
print("#"*20)

Running simulation with 4 MPI processes
Simulation command:
export OMP_NUM_THREADS=2; export OMP_PROC_BIND=spread; export OMP_PLACES=threads; mpirun -np 4 singularity exec /home/lmascare/bench/docker/images/bench.sif pdirun /home/lmascare/bench/simulation/build/main /home/lmascare/bench/simulation/setup.ini /home/lmascare/bench/simulation/io_deisa.yml --kokkos-map-device-id-by=mpi_rank > /home/lmascare/bench/experiment/simulation.e 2>&1
Output:

--------------------
Error:
Connection to dahu-9.grenoble.grid5000.fr closed.


####################
